In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preparation

---

## Data Extraction

In [16]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [17]:
sample_submission.shape

In [26]:
train.shape

In [27]:
train.info()

In [19]:
test.shape

In [28]:
test.info()

## Data Concatenation

In [20]:
data = pd.concat([train, test], sort = False)
data.shape

In [24]:
data.info()

In [21]:
data.head()

## Null Check

In [22]:
null_cols = [col for col in data.iloc[: , : -1].columns if data[col].isnull().sum() != 0]
null_cols

In [23]:
float_cols = [col for col in data.iloc[: , 1 : -1].columns if data[col].dtype == "float64"]
len(float_cols)